In [26]:
import pandas as pd
import numpy as np

In [19]:
bakrep_metadata = pd.read_csv("data/bakrep-export_18112024.tsv",sep='\t', low_memory=False, index_col="#id")

In [20]:
[tup for tup in zip(bakrep_metadata.columns, bakrep_metadata.dtypes)]

[('bakta.genome.genus', dtype('O')),
 ('bakta.genome.species', dtype('O')),
 ('bakta.genome.strain', dtype('O')),
 ('bakta.stats.no_sequences', dtype('float64')),
 ('bakta.stats.size', dtype('float64')),
 ('bakta.stats.gc', dtype('float64')),
 ('bakta.stats.n_ratio', dtype('float64')),
 ('bakta.stats.n50', dtype('float64')),
 ('bakta.stats.coding_ratio', dtype('float64')),
 ('gtdbtk.classification.domain', dtype('O')),
 ('gtdbtk.classification.phylum', dtype('O')),
 ('gtdbtk.classification.class', dtype('O')),
 ('gtdbtk.classification.order', dtype('O')),
 ('gtdbtk.classification.family', dtype('O')),
 ('gtdbtk.classification.genus', dtype('O')),
 ('gtdbtk.classification.species', dtype('O')),
 ('mlst.sequence_type', dtype('O')),
 ('checkm2.quality.completeness', dtype('float64')),
 ('checkm2.quality.contamination', dtype('float64')),
 ('metadata.accession', dtype('O')),
 ('metadata.collected_by', dtype('O')),
 ('metadata.collection_date', dtype('O')),
 ('metadata.country', dtype('O'))

In [21]:
bakrep_metadata[["metadata.accession"]]

,metadata.accession
#id,
SAMD00000344,SAMD00000344
SAMD00000345,SAMD00000345
SAMD00000355,SAMD00000355
SAMD00000550,SAMD00000550
SAMD00000552,SAMD00000552
...,...
SAMN10405967,SAMN10405967
SAMN10405968,SAMN10405968
SAMN10407504,SAMN10407504


In [22]:
est_card = pd.read_csv('/Users/ktruong/data/661k/est_card_661k.csv', header = None, names= ['id','card'], index_col= 'id' )

In [23]:
est_card

,card
id,
SAMN02844126,4722564.00
SAMN04558116,1589918.00
SAMEA2068521,6482382.50
SAMN09281158,2937214.75
SAMN09843229,4991481.50
...,...
SAMEA3481361,2161193.00
SAMN09650548,5065128.50
SAMN09463236,5239581.00


In [37]:
# missing 3 genomes
set(est_card.index.values) - set(bakrep_metadata.index.values)

{'SAMD00080514', 'SAMD00080519', 'SAMD00080520'}

In [52]:
species_groups =  bakrep_metadata.groupby(['gtdbtk.classification.species']).groups
group_sizes = bakrep_metadata.groupby(['gtdbtk.classification.species']).size()

In [58]:
group_sizes.sort_values()

gtdbtk.classification.species
1-14-0-20-39-49 sp002787635         1
Pantoea brenneri                    1
Pantoea anthophila                  1
Pantoea allii                       1
Pannonibacter phragmitetus          1
                                ...  
Staphylococcus aureus           47997
Mycobacterium tuberculosis      49020
Streptococcus pneumoniae        51433
Escherichia coli                89136
Salmonella enterica            178585
Length: 8207, dtype: int64

In [84]:
group_sizes_small = group_sizes[group_sizes<100]
len(group_sizes_small)

8067

In [85]:
group_sizes_small.sort_values()

gtdbtk.classification.species
1-14-0-20-39-49 sp002787635        1
Paraburkholderia kururiensis       1
Paraburkholderia dilworthii        1
Paraburkholderia diazotrophica     1
Paraburkholderia azotifigens       1
                                  ..
Ligilactobacillus salivarius      96
Elizabethkingia anophelis         96
Escherichia marmotae              97
Deinococcus radiodurans           98
Chlamydophila pecorum             98
Length: 8067, dtype: int64

In [105]:
group_sizes[(group_sizes>100) & (group_sizes<1000)].sort_values()

gtdbtk.classification.species
Enterobacter kobei              104
Bacillus_A paranthracis         105
Aeromonas veronii               106
Moraxella catarrhalis           107
Salmonella bongori              107
                               ... 
Mycobacterium intracellulare    880
Mycobacterium marinum           891
Bacillus_A anthracis            893
Neisseria lactamica             951
Salmonella houtenae             972
Length: 104, dtype: int64

In [78]:
group_sizes.sort_values(ascending=False).head(20)

gtdbtk.classification.species
Salmonella enterica           178585
Escherichia coli               89136
Streptococcus pneumoniae       51433
Mycobacterium tuberculosis     49020
Staphylococcus aureus          47997
Campylobacter_D jejuni         28336
Neisseria meningitidis         17153
Streptococcus pyogenes         16821
Klebsiella pneumoniae          13637
Clostridioides difficile       13579
Listeria monocytogenes         12245
Listeria monocytogenes_B       12244
Streptococcus agalactiae       10359
Neisseria gonorrhoeae           8904
Campylobacter_D coli            8849
Enterococcus_B faecium          8284
Pseudomonas aeruginosa          6286
Vibrio cholerae                 5562
Acinetobacter baumannii         5124
Mycobacterium abscessus         2689
dtype: int64

In [79]:
species_groups_bakta =  bakrep_metadata.groupby(['bakta.genome.species']).groups
group_sizes_bakta = bakrep_metadata.groupby(['bakta.genome.species']).size()

In [81]:
group_sizes_bakta.sort_values(ascending=False).head(20)

bakta.genome.species
enterica         177020
coli              81894
pneumoniae        65831
tuberculosis      48945
aureus            48357
jejuni            27491
monocytogenes     24676
meningitidis      16641
pyogenes          16622
difficile         13466
agalactiae        10330
gonorrhoeae        8797
faecium            8429
sp.                8216
sonnei             7583
aeruginosa         6168
cholerae           5906
baumannii          5127
flexneri           4904
abscessus          2694
dtype: int64